In [2]:
import re
import pdfplumber
import pandas as pd
from collections import namedtuple
#https://www.youtube.com/watch?v=syEfR1QIGcY

In [54]:
Line = namedtuple("Line","CID10  DESC JANEIRO FEVEREIRO MARÇO ABRIL MAIO JUNHO JULHO AGOSTO SETEMBRO OUTUBRO NOVEMBRO DEZEMBRO")
#Line = namedtuple("Line","CID10 DESC")

In [52]:
line_re =re.compile(r'(\w\d{2}) ([\s\S]+(?:[A-Za-z])) ([\s\S]+(?:[0-9 -]))')
#line_re.search('A15 Tuberculose respiratória, com confirmação bacteriológica e his 47 49 42 29 36 36 41 26 38 35 32 37').group(3)
line_re.search('A00 Cólera F ––––––– –– – –').group(3)

'––––––– –– – '

In [53]:
file='data//2008_Acidentarios.pdf'

In [58]:
lines =[]
total_check =0 
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        for line in text.split('\n'):
            comp =line_re.search(line)
            if comp:
                CID, CID_DES, DADOS = comp.group(1), comp.group(2) , comp.group(3)
                #print(CID,CID_DES)
                print(DADOS)
                #lines.append(Line(CID,CID_DES,DADOS))

     20          9          11         14         17         17           2 2         20           1 5            1 9
       2 5          2 2         30         34         31         21         18         25           3 1         31           3 5            3 9
       5 5          4 7         52         61         41         52         57         68           7 1         81           4 9            5 7
         8            10         14          8            5         16          7            7           1 1         15             7               7 
         8             9          14          7            5         16          6            7             9          15             6               7 
– – – – – – – –             1  –             1  
–            1  – – – – – – – – – 
– – –          1  – –          1  – – – – 
– – – – – – – –             1  – – 
     3 68        2 99       358       388       322       349       388       377         3 98       449         4 04          

In [210]:
df = pd.DataFrame(lines)
df.head()
df

,CID10,DESC
0,A09,Doenças infecciosas intestinais
1,A00,Cólera
2,A01,Febres tifóide e paratifóide
3,A02,Outras infecções por Salmonella
4,A03,Shiguelose
...,...,...
2162,Z95,Presença de implantes e enxertos cardíacos e v...
2163,Z96,Presença de outros implantes funcionais
2164,Z97,Presença de outros dispositivos protéticos
2165,Z98,Outros estados pós-cirúrgicos
